In [1]:
import nltk
import string
import numpy as np

from collections import Counter
from abc import ABCMeta, abstractmethod

from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import sent_tokenize, wordpunct_tokenize

In [2]:
text = "В конце мая наплыв студентов в США по всемирно известной программе международного обмена достигает максимальной интенсивности .\nИз многих интернет источников , таких как социальная сеть для участников Work &amp; Travel USA , статистически замечено , что в 2010 году вакансии для зарубежных студентов существенно сократились в количестве .\nМногие из приехавших ребят ищут работу уже по месту , бродя по американским мегаполисам , заходя в каждый ресторанчик в поисках должностей официантов , барменов и всего , что предложат американские работодатели .\nОбоснованием фактора увеличения безработицы является влияние кризиса в США 2009 года .\nНо , несмотря ни на что , студенты со всей Европы продолжают ездить в Соединенные Штаты во время летних каникул , и с каждым годом количество участников увеличивается .\nЭто дает возможность предположить , что большинство участников программы заинтересовано не в заработке денег , а в возможности попутешествовать по всемирно известным местам исторического значения , музеям , заповедникам , городам .\nМногим интересно узнать о способе и стиле жизни американской нации .\nСледовательно , основная цель и предназначение международной программы Work&amp;Travel USA выполняются .\nВзаимный обмен культурными знаниями среди молодежи оказывает хорошее влияние на развитие цивилизации в глобальном плане , сближает культуры и понимание наций всего мира .\nЭто в общей идее напоминает одну из функций Организации Объединенных Наций по созданию всемирного дружного союза .\nПо прошествии некоторого времени , современная молодежь займет позиции чиновников и управляющих во всех государствах мира .\nСближение и ознакомление в юном возрасте со многими странами даст выигрыш в более дружном международном отношении и , вполне вероятно , более прогрессивном разоружении ядерных стран мира , что также является одним из направлений деятельности ООН .\nНесмотря на всю прелесть программ такого типа , посольство США в 2010 году ужесточило отбор студентов на право получения визы .\nОтзывы студентов о прохождении посольства можно почитать в группе &quot; Посольство США &quot; специализированной социальной сети .\nПравительство США опасается массовой нелегальной миграции иностранных студентов с помощью программы Work&amp;Travel USA .\nПоэтому были ужесточены критерии отбора для студентов последних курсов , рассматриваемых как потенциальных нелегалов .\nВ этом году для студентов 4 и 5 курсов высших учебных заведений необходимо было предоставить справки о продолжении обучения в сентябре 2010 года для подтверждения факта возврата студента в свою страну по окончании программы .\nНа данный момент количество нелегально оставшихся в Соединенных Штатах участников программ международного обмена исчисляется сотнями .\nВ подтверждение этому есть аудио-интервью у одного из таких эмигрантов в Нью-Йорке в одном из подкастов проекта &quot; Молодежная Заграница &quot; .\n\n&quot; Эта программа на самом деле потрясающая .\nСтуденты лучше начинают говорить по-английски , узнают о США , знакомятся с американцами , американцы узнают от них о России , у них появляются новые друзья .\nНам кажется , что это очень ценная программа .\nНа личном опыте : я помню , что мы с семьей отдыхали пару лет назад на пляже в Северной Каролине .\nОстановились пообедать .\nИ все , кто обслуживал клиентов , оказались родом откуда-то из Сибири .\nМне было очень приятно поговорить с ними по-русски .\nПохоже было , что студенты были довольны тем , как они проводят лето . &quot;\n\nГоворит генеральный консул США , Курт Аменд , в интеренет издании Газета.ру\n\n\n"

# Tools

In [3]:
PUNKT = set(string.punctuation + "«»№_—")
STOPWORDS = set(stopwords.words("russian"))

stemmer = RussianStemmer("stemmer")

    
def text_to_sentences(text):
    return [sentence for sentence in sent_tokenize(text)]


def is_punkt(word):
    return all(char in PUNKT for char in word) or word in STOPWORDS


def sentence_to_tokens(sentence):
    return [stemmer.stem(word) for word in wordpunct_tokenize(sentence) if not is_punkt(word)]
 
    
def parse(text):
    for sentence in text_to_sentences(text):
        yield sentence_to_tokens(sentence)

        
def get_index(values, p):
    return len(values[np.cumsum(values / sum(values)) < p])


def is_allowed(ch):
    return (ch >= 'а' and ch <= 'я') or (ch >= 'А' and ch <= 'Я') or ch == ' '


def filter_text(text):
    return ''.join(char for char in text if is_allowed(char))

In [4]:
class Summarizer(metaclass=ABCMeta):
    @abstractmethod
    def summarize(text):
        pass

# Approach 1

In [5]:
class NaiveSummarizer(Summarizer):
    def __init__(self, max_len: int = 300):
        self.max_len = max_len
    
    
    def summarize(self, text):
        return text[:self.max_len]

# Approach 2

In [6]:
class FreqSummarizer(Summarizer):
    def __init__(self, max_cnt: int = 5, window_size: int = 7):
        self.max_cnt = max_cnt
        self.size = window_size
    
    
    def summarize(self, text):
        vocab = self._get_stem_vocab(text)
        sentences = text_to_sentences(text)[:self.max_cnt]
        ranking = sorted(sentences, key=lambda it: self._evalutate(it, vocab), reverse=True)
        return ''.join(ranking)[:300]
    
    
    def _evalutate(self, sentence, vocab):
        tokens = sentence_to_tokens(sentence)
        
        score = 0
        
        for i in range(0, len(tokens) - self.size + 1):
            cnt = sum(token in vocab for token in tokens[i : i + self.size])
            score += 2 ** cnt - 1
            
        return score
    
    
    @staticmethod
    def _get_stem_vocab(text, l=0.15, r=0.85):
        tokens = [token for sentence in parse(text) for token in sentence]
        most_common = Counter(tokens).most_common()
        tokens = np.array(list(map(lambda it: it[0], most_common)))
        cnt = np.array(list(map(lambda it: it[1], most_common)))

        left_index = get_index(cnt, l) + 1
        right_index = get_index(cnt, r) + 1

        return set(tokens[left_index:right_index])

In [7]:
summarizer = FreqSummarizer()

In [8]:
filter_text(summarizer.summarize(text))

'Многие из приехавших ребят ищут работу уже по месту  бродя по американским мегаполисам  заходя в каждый ресторанчик в поисках должностей официантов  барменов и всего  что предложат американские работодатели Из многих интернет источников  таких как социальная сеть для участников   '